# 04. Agents, Function Calling, Text-to-SQL, MCP - 주석 포함 버전

In [3]:
# Function Calling 의사코드
schema = {
    "name": "get_weather",
    "parameters": {"type":"object","properties":{"city":{"type":"string"}}}
}

def get_weather(city):
    # 실제 코드에서는 requests.get()을 이용해 외부 API 호출
    return {"city": city, "temp": 26, "desc": "cloudy"}

# LLM이 함수 호출 의도를 판단 → get_weather(city) 수행 → 응답 조합


In [ ]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "주어진 도시의 현재 날씨를 조회합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "도시명(한국어/영어 모두 허용). 예: 'Seoul' 또는 '서울'"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        },
    }
]


In [ ]:
def get_weather(city: str) -> dict:
    # 실제 코드에서는 외부 API 호출 (예: OpenWeather)
    # 예시는 "의사(doctor)가 쓴 듯한" 대강 코드가 아니라, 타입/키를 명확히!
    return {"city": city, "temp": 26, "desc": "Cloudy"}


In [8]:

import json
from openai import OpenAI


client = OpenAI()  # 환경변수 OPENAI_API_KEY 필요

messages = [
    {"role": "system", "content": "너는 도움이 되는 어시스턴트야. 필요하면 툴(get_weather)을 호출해."},
    {"role": "user", "content": "오늘 서울 날씨 어때?"}
]

# 1) 1차 호출: 도구 정의(tools)를 넘기고, 모델이 '필요시' 함수 호출을 결정하도록 함
first = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # 모델이 스스로 호출 여부 판단
)

assistant_msg = first.choices[0].message
messages.append({"role": "assistant", "content": assistant_msg.content or "", "tool_calls": assistant_msg.tool_calls})

# 2) 모델이 함수 호출을 결정했는지 확인
if assistant_msg.tool_calls:
    for call in assistant_msg.tool_calls:
        if call.type == "function" and call.function.name == "get_weather":
            args = json.loads(call.function.arguments or "{}")
            city = args.get("city")
            tool_result = get_weather(city)

            # 툴 결과를 'tool' 역할로 대화에 추가 (id는 tool_call_id에 바인딩)
            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "content": json.dumps(tool_result, ensure_ascii=False)
            })

    # 3) 2차 호출: 툴 결과를 본 모델이 최종 자연어 응답 생성
    second = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    final_text = second.choices[0].message.content
else:
    # 함수 호출이 불필요하다고 판단한 경우(바로 답변)
    final_text = assistant_msg.content

print(final_text)


오늘 서울의 날씨는 흐림이며, 기온은 26도입니다. 외출할 때 참고하세요!


In [4]:
# Text-to-SQL 의사코드
schema = {"tables":["faq"], "columns":{"faq":["question","answer"]}}

q = "신청 절차를 알려줘"
sql = "SELECT answer FROM faq WHERE question LIKE '%신청 절차%';"
print('SQL:', sql)


SQL: SELECT answer FROM faq WHERE question LIKE '%신청 절차%';


In [9]:
# ==========================================
# Text-to-SQL: One-cell MWE for Jupyter
# ==========================================
import os, json, sqlite3, traceback
from pathlib import Path
from openai import OpenAI

# --- 0) 사전 준비 ------------------------------------------------------------
assert os.getenv("OPENAI_API_KEY"), "환경변수 OPENAI_API_KEY가 없습니다."

DB_PATH = "t2s_demo.db"

def init_demo_db(db_path: str = DB_PATH):
    """데모용 SQLite DB 생성(존재 시 건너뜀)."""
    if Path(db_path).exists():
        return
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # 샘플 테이블: customers, orders
    cur.executescript("""
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        region TEXT NOT NULL
    );
    CREATE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        order_date TEXT NOT NULL,
        amount REAL NOT NULL,
        FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
    );
    """)
    cur.executemany("INSERT INTO customers(customer_id, name, region) VALUES (?, ?, ?)", [
        (1, "Alice", "Seoul"),
        (2, "Bob",   "Busan"),
        (3, "Cathy", "Seoul"),
    ])
    cur.executemany("INSERT INTO orders(order_id, customer_id, order_date, amount) VALUES (?, ?, ?, ?)", [
        (1001, 1, "2025-01-10", 120.0),
        (1002, 1, "2025-02-05",  80.5),
        (1003, 2, "2025-02-20", 250.0),
        (1004, 3, "2025-03-01",  99.9),
        (1005, 2, "2025-03-11", 300.0),
    ])
    conn.commit()
    conn.close()

init_demo_db()

# --- 1) 안전한 SQL 실행 유틸 ---------------------------------------------------
ALLOWED_SQL_PREFIXES = ("SELECT", "WITH")  # 읽기 전용 허용

def is_safe_sql(sql: str) -> bool:
    if not isinstance(sql, str): 
        return False
    # 간단한 가드: 앞부분 공백 제거 후 허용된 키워드로 시작
    return sql.strip().upper().startswith(ALLOWED_SQL_PREFIXES)

def fetch_all(sql: str, params: tuple = ()):
    assert is_safe_sql(sql), f"허용되지 않은 SQL입니다: {sql[:80]}..."
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(sql, params)
    rows = cur.fetchall()
    cols = [d[0] for d in cur.description] if cur.description else []
    conn.close()
    return {"columns": cols, "rows": [dict(r) for r in rows]}

def list_tables():
    q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
    return fetch_all(q)

def describe_table(table: str):
    # PRAGMA table_info: 컬럼 스키마 확인
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table})")
    rows = cur.fetchall()
    cols = ["cid", "name", "type", "notnull", "dflt_value", "pk"]
    data = [dict(zip(cols, r)) for r in rows]
    conn.close()
    return {"table": table, "columns": data}

# --- 2) Tools(JSON Schema; LLM Function Calling 규격) -------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_tables",
            "description": "데이터베이스에 존재하는 테이블 목록을 반환합니다.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "describe_table",
            "description": "특정 테이블의 컬럼 스키마를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "table": {"type": "string", "description": "테이블명"}
                },
                "required": ["table"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "run_sql_select",
            "description": "SELECT/WITH 쿼리를 실행하고 결과를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql": {
                        "type": "string",
                        "description": "SELECT 또는 WITH로 시작하는 읽기 전용 SQL"
                    }
                },
                "required": ["sql"],
                "additionalProperties": False
            }
        }
    }
]

# --- 3) 서버측(백엔드) 실제 함수 바인딩 ---------------------------------------
def run_sql_select(sql: str):
    return fetch_all(sql)

SERVER_FUNCS = {
    "list_tables": lambda **_: list_tables(),
    "describe_table": lambda table, **_: describe_table(table),
    "run_sql_select": lambda sql, **_: run_sql_select(sql),
}

# --- 4) LLM 루프: 스키마 파악→SQL 생성→실행→최종 응답 ------------------------
client = OpenAI()

def t2s_chat(user_question: str, max_rounds: int = 4):
    """
    Text-to-SQL 대화 루프.
    모델이 필요시 list_tables/describe_table 호출 → SQL 생성 → run_sql_select 호출.
    """
    messages = [
        {"role": "system", "content": (
            "너는 Text-to-SQL 전문가야. 읽기 전용 쿼리만 작성해. "
            "테이블/컬럼이 확실치 않다면 먼저 스키마 도구(list_tables/describe_table)로 확인한 뒤 "
            "정확한 SQL을 만든 후 run_sql_select를 호출해. "
            "SQL은 SQLite 호환으로 작성하고, 결과 요약을 한국어로 해."
        )},
        {"role": "user", "content": user_question}
    ]

    for _ in range(max_rounds):
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        msg = resp.choices[0].message
        messages.append({
            "role": "assistant",
            "content": msg.content or "",
            "tool_calls": msg.tool_calls
        })

        # 툴 호출 없으면 최종 응답
        if not msg.tool_calls:
            return msg.content

        # 툴 호출 처리
        for call in msg.tool_calls:
            if call.type != "function":
                continue
            fn = call.function.name
            args = json.loads(call.function.arguments or "{}")

            if fn not in SERVER_FUNCS:
                tool_content = json.dumps({"error": f"unknown tool: {fn}"}, ensure_ascii=False)
            else:
                try:
                    # 서버측 함수 실행
                    res = SERVER_FUNCS[fn](**args)
                    tool_content = json.dumps(res, ensure_ascii=False)
                except Exception:
                    tool_content = json.dumps(
                        {"error": "tool execution failed", "trace": traceback.format_exc()},
                        ensure_ascii=False
                    )

            # tool 역할 메시지로 결과 반영
            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "content": tool_content
            })

    # 라운드 초과 시
    return "요청을 처리하는 중 반복 한도를 초과했습니다."

# --- 5) 데모 질의 --------------------------------------------------------------
tests = [
    "테이블 뭐 있어?",  # list_tables 호출 유도
    "customers 테이블 컬럼 알려줘",  # describe_table 호출
    "지역(region)별 고객 수를 구해줘",  # GROUP BY
    "월별 총 주문금액을 구해줘",       # strftime를 이용한 월 단위 집계
    "서울(Seoul) 지역 고객의 총 구매액은?"  # join + filter
]

for q in tests:
    print("\nQ:", q)
    print("A:", t2s_chat(q))



Q: 테이블 뭐 있어?
A: 데이터베이스에는 "customers"와 "orders"라는 두 개의 테이블이 있습니다.

Q: customers 테이블 컬럼 알려줘
A: customers 테이블의 컬럼은 다음과 같습니다:

1. **customer_id** (INTEGER, 기본키)
2. **name** (TEXT, NOT NULL)
3. **region** (TEXT, NOT NULL)

Q: 지역(region)별 고객 수를 구해줘
A: 각 지역별 고객 수는 다음과 같습니다:

- 부산: 1명
- 서울: 2명

Q: 월별 총 주문금액을 구해줘
A: 월별 총 주문금액은 다음과 같습니다:

- 2025년 1월: 120.0
- 2025년 2월: 330.5
- 2025년 3월: 399.9

Q: 서울(Seoul) 지역 고객의 총 구매액은?
A: 서울 지역 고객의 총 구매액은 300.4입니다.
